In [1]:
!nvidia-smi

Tue Jun 24 15:41:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050        Off |   00000000:01:00.0 Off |                  N/A |
| N/A   60C    P0             10W /   30W |       7MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelarate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate



ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]2 [accelerate]


In [10]:
!pip install evaluate

In [12]:
#now implementing transformers


""""
from transformers import pipeline, set_seed

 # pipeline: A Hugging Face utility to quickly use pretrained models for tasks like summarization, translation, etc.
 #set_seed: Ensures reproducibility by setting a fixed seed for random operations.


#______________________________________________________
#load_dataset: Loads datasets from Hugging Face's online hub.
#load_from_disk: Loads a dataset saved locally (e.g., after preprocessing and saving to disk).




from datasets import load_dataset,load_from_disk
#____________________________________________________
import matplotlib.pyplot as plt
#
import pandas as pd
#------------------


from datasets import load_dataset,load_metric #Redundant load_dataset again.
#load_metric: Loads evaluation metrics (e.g., ROUGE, BLEU) used in NLP tasks.
#(Note: load_metric is deprecated in favor of evaluate.load.)





#---------------------------------------
# Model and Tokenizer

#AutoModelForSeq2SeqLM: Loads any sequence-to-sequence model (like T5, BART) for tasks like summarization, translation.

#AutoTokenizer: Loads the tokenizer corresponding to the model.


from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
#------------------------------------------------------

import nltk #nltk: Python library for text processing.


from nltk.tokenize import sent_tokenize #sent_tokenize: Splits text into sentences (useful in pre-processing for summarization).


#Adds progress bars to loops (great for monitoring large dataset/model operations).
from tqdm import tqdm
import torch #... loading pytorch
#Downloads the pretrained Punkt tokenizer models (used by sent_tokenize).



nltk.download("punkt")
"""

# Install required libraries (uncomment if running for the first time)
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr matplotlib pandas torch tqdm nltk evaluate

# Hugging Face Transformers pipeline and reproducibility
from transformers import pipeline, set_seed

# Hugging Face Datasets: load datasets from the hub or disk
from datasets import load_dataset, load_from_disk

# For evaluation metrics (ROUGE, BLEU, etc.)
from evaluate import load as load_metric

# Plotting and data handling
import matplotlib.pyplot as plt
import pandas as pd

# Model and Tokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# Text processing
import nltk
from nltk.tokenize import sent_tokenize

# Progress bars and PyTorch
from tqdm import tqdm
import torch

# Download NLTK punkt tokenizer if not already present
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/divyansh-
[nltk_data]     rawat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail" # this model we use for summarization
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Error while downloading from https://cdn-lfs.hf.co/google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1750763940&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDc2Mzk0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Ie8IV%7E7UDIPIq98R90HuILyLOGCg-0qMiAKHiI9e6q0uDI8H5yCzBfIP9yzeAs7PDqQMLSfgcVIrsU1xLsfRm6klLEHe%7EmhV0Wr5OT%7ENg3yUzGyX-bx91YQhahrf8X8DE2MtvcPayB%7EbBOq20aPTDbR3DNbPbzRHFnDXo3BMyUGXnziOvfNJLbWTU4-lfxl1ITjQTiMP02ADL8paRL0GW%7Ew30R5gENLBUQvfcgQI6XU8baerkM33S3KemkRTCELM8C6mduOZjWDPzoZcv1-xMbVS7P-4mSha

In [ ]:
#dataset using
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip


--2025-06-24 16:09:49--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2025-06-24 16:09:50--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  1.28MB/s    in 12s     

2025-06-24 16:10:03 (662 KB/s) - ‘summarizer-data.zip.1’

In [ ]:
from datasets import load_from_disk
dataset_samsum = load_from_disk('file://./samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]




print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])
#

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )


    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [ ]:
dataset_samsum_pt["train"]

In [ ]:
#doing training now
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:
from transformers import TrainingArguments,Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    eval_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16



)

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Splitting the dataset into smaller batches for processing.
    Yields successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        '''     parameter for length penalty ensures that model does not generate sequence    '''

        #finally , we decode the generated texts,
        #replace the token =, and add the decoded text with the references to the metric

        decoded_summaries = [tokenizer.decode(s,skip_special_tokens = True,
                                             clean_up_tokenization_spaces = True)
               for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #finalyy compute and return the Rouge scores

    score = metric.compute()
    return score



In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )
#

In [ ]:
##saving the model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
##saving tokens
tokenizer.save_pretrained("tokenizer")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")  #load

In [ ]:
#prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])